In [ ]:
import subprocess
import os
from google.colab import drive
import pylandstats as pls
import pandas as pd
import rasterio
import folium
import numpy as np
from matplotlib.colors import ListedColormap

# Function to check and install necessary packages
def setup_environment():
    def check_install(package):
        try:
            __import__(package)  # Try to import the package
            print(f'{package} already installed.')
        except ImportError:
            print(f'{package} not installed. Installing {package}...')
            subprocess.check_call(['pip', 'install', package])  # Install the package if not found

    # List of necessary packages
    packages = ['pylandstats', 'openpyxl', 'rasterio', 'folium', 'pandas', 'numpy']

    # Check and install each package
    for package in packages:
        check_install(package)

# Execute setup_environment function to ensure required packages are installed
setup_environment()

# Function to calculate and save landscape metrics and plot the raster interactively
def calculate_save_metrics_and_plot(input_file, output_folder, output_base_name, color_excel, nodata_value=0):
    try:
        # Load raster data and calculate metrics using pylandstats
        print("Loading raster data and calculating metrics...")
        ls = pls.Landscape(input_file, nodata=nodata_value)
        metrics_df = ls.compute_class_metrics_df().to_csv(os.path.join(output_folder, f'{output_base_name}_metrics_output.csv'), sep='\t', encoding='utf-8')
        print(f"Metrics calculated and saved to {os.path.join(output_folder, f'{output_base_name}_metrics_output.csv')}")

        # Read color legend Excel file using pandas
        print("Reading color legend Excel file...")
        color_df = pd.read_excel(color_excel)
        color_map = {row['Class_ID']: row['Color'] for _, row in color_df.iterrows()}

        # Load raster data using rasterio
        print("Loading raster data using rasterio...")
        with rasterio.open(input_file) as src:
            bounds = src.bounds
            raster_array = src.read(1)
            transform = src.transform

        # Create a custom colormap based on colors from the legend
        print("Creating custom colormap...")
        unique_classes = np.unique(raster_array)
        colormap = np.zeros((256, 4), dtype=int)
        for class_id, color in color_map.items():
            hex_color = color.lstrip('#')
            rgb = tuple(int(hex_color[i:i + 2], 16) for i in (0, 2, 4))
            colormap[class_id] = (*rgb, 255)  # Add alpha channel

        colormap = ListedColormap([tuple(c / 255 for c in colormap[i]) for i in range(256)])

        # Create folium map centered on raster data bounds
        print("Creating folium map...")
        m = folium.Map(location=[(bounds.top + bounds.bottom) / 2, (bounds.left + bounds.right) / 2], zoom_start=10)

        # Add raster data overlay to the map
        print("Adding raster data overlay to the map...")
        folium.raster_layers.ImageOverlay(
            image=raster_array,
            bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
            colormap=lambda x: colormap.colors[x],
            opacity=0.6
        ).add_to(m)

        # Add zoom control to the map
        print("Adding zoom control to the map...")
        folium.LayerControl().add_to(m)

        print("Map creation complete.")
        # Display the map
        return m

    except FileNotFoundError as fe:
        print(f"File not found error: {fe}")
    except ValueError as ve:
        print(f"Value error: {ve}")
    except Exception as e:
        print(f"Error during processing: {e}")

# Prompt the user to input the output folder path
output_folder = input('Enter the path to the output folder where results will be saved: ')

# Check if the provided output folder path exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created output folder: {output_folder}")

# Prompt the user to input the file paths and output base name
input_raster = input('Enter the path to the input raster (.tif) file: ')
color_excel = input('Enter the path to the color legend Excel (.xlsx) file: ')
output_base_name = input('Enter the base name for output files (without extension): ')

# Check if the provided files exist and have correct extensions
if not os.path.isfile(input_raster) or not input_raster.lower().endswith('.tif'):
    print(f"Error: No .tif file found at the specified path or incorrect file type: {input_raster}")
    exit()

if not os.path.isfile(color_excel) or not color_excel.lower().endswith('.xlsx'):
    print(f"Error: No .xlsx file found at the specified path or incorrect file type: {color_excel}")
    exit()

print(f'Input raster: {input_raster}')
print(f'Color Excel: {color_excel}')
print(f'Output folder: {output_folder}')
print(f'Output base name: {output_base_name}')

# Call the function to perform calculations, save metrics, and plot raster
m = calculate_save_metrics_and_plot(input_raster, output_folder, output_base_name, color_excel)

# Save the interactive map as HTML
html_output = os.path.join(output_folder, f'{output_base_name}_map_output.html')
try:
    print("Saving the map as HTML...")
    m.save(html_output)
    print(f'Map saved to {html_output}')
except Exception as e:
    print(f"Error saving map: {e}")

# Print completion message
print('Process completed successfully.')

# Display the interactive map in Jupyter Notebook or Google Colab
m
